##### imports

In [276]:
from numpy import exp
from numpy import sqrt
from numpy import cos
from numpy import e
from numpy import pi
from typing import List, Callable, Tuple
import random

#### Definindo váriaveis e função de avaliação

In [277]:
A = 20
B = 0.2
C = 2 * pi
D = 30

In [278]:
TAM_DA_POPULACAO = 10
POPULACAO = []
QTD_DE_FILHOS = 7 * TAM_DA_POPULACAO
TAXA_APRENDIZAGEM = 1 / sqrt(D)
EPSILON_0 = 0.001
MAX_ITERS = 10_000
CURR_SUCCESS = 0

In [279]:
def ackley_funct(x: List[float]):
  return - A * exp(-B * sqrt(sum(map(lambda n: n ** 2, x)) / D)) \
          - exp(sum(map(lambda n: cos(C * n), x)) / D) \
          + A + exp(1)

In [280]:
FITNESS_FUNC = ackley_funct

In [281]:
def fitness(fitness_func: Callable, cromossome: List[float]):
    return fitness_func(cromossome)

In [282]:
print(f'{ackley_funct([0 for _ in range(30)]) : .15f}')

 0.000000000000000


# Estratégia Evolutiva

## Versão 1

### Inicialização da população

In [283]:
def generate_individual():
  x = []
  for i in range(D):
    x.append(random.uniform(-15, 15))
    
  return x

In [284]:
def generate_population():
  global POPULACAO, TAM_DA_POPULACAO
  
  for _ in range(TAM_DA_POPULACAO):
    POPULACAO.append((generate_individual(), random.normalvariate(0, 1)))

In [285]:
def reset_population() -> None:
    global POPULACAO

    POPULACAO = []

### Seleção de pais

In [286]:
def select_parents():
    global POPULACAO

    pai1, pai2 = random.sample(POPULACAO, 2)

    return pai1, pai2

### Recombinação

In [287]:
# Local discreto
def evolutive_crossover(pai1: Tuple[List[float], float], pai2: Tuple[List[float], float]):
    filho = []
    for i in range(D):
        filho += random.sample([pai1[0][i], pai2[0][i]], 1)
        
    sigma = random.sample([pai1[1], pai2[1]], 1)[0]

    return filho, sigma

In [288]:
def crossover():
    global QTD_DE_FILHOS

    filhos = []
    for _ in range(QTD_DE_FILHOS):
        pai1, pai2 = select_parents()
        filhos.append(evolutive_crossover(pai1, pai2))

    return filhos

### Mutação

In [289]:
def regra_1_quinto(sigma: float, sucesso: float) -> float:
    c = 0.8
    
    if sucesso > 1 / 5:
        sigma = sigma / c
    elif sucesso < 1 / 5:
        sigma = sigma * c
    
    return sigma

In [290]:
def calc_sigma(sigma: float) -> float:
    global TAXA_APRENDIZAGEM, EPSILON_0

    new_sigma = sigma * exp(TAXA_APRENDIZAGEM * random.uniform(0, 1))

    return max(new_sigma, EPSILON_0)

In [291]:
def test_success(old_indiv_fitness, new_indiv_fitness):
    global CURR_SUCCESS

    if new_indiv_fitness <= old_indiv_fitness:
        CURR_SUCCESS += 1

In [292]:
def get_success() -> float:
    global CURR_SUCCESS
    
    return CURR_SUCCESS / QTD_DE_FILHOS

In [293]:
def reset_success() -> None:
    global CURR_SUCCESS

    CURR_SUCCESS = 0

In [294]:
def mutate_and_calc_sucess(individuo: List[float], sigma: float):
    novo_sigma = calc_sigma(sigma)

    old_fitness = fitness(FITNESS_FUNC, individuo)
    
    for i in range(len(individuo)):
        individuo[i] = min(max(individuo[i] + novo_sigma * random.normalvariate(0, 1), -15), 15)

    new_fitness = fitness(FITNESS_FUNC, individuo)
    test_success(old_fitness, new_fitness)

    return individuo, novo_sigma

In [295]:
def mutate(filhos: List[Tuple[List[float], float]]):
    global QTD_DE_FILHOS

    filhos_mutados = []
    for i in range(QTD_DE_FILHOS):
        filhos_mutados.append(mutate_and_calc_sucess(filhos[i][0], filhos[i][1]))

    return filhos_mutados

### Seleção de sobreviventes

#### Geracional

In [308]:
def generational(filhos: List[Tuple[List[float], float]]):
    global POPULACAO, FITNESS_FUNC, TAM_DA_POPULACAO
    
    filhos.sort(key=lambda filho: fitness(FITNESS_FUNC, filho[0]), reverse=False)
    POPULACAO = filhos[:TAM_DA_POPULACAO]

### Loop

In [297]:
def is_close(a: float, b: float, diff: float = 0.00001):
    if a + diff > b and b + diff > a:
        return True
        
    return False

In [298]:
def should_stop(curr_best_fitness: float, curr_iter: int):
    if is_close(curr_best_fitness, 0):
        return True
    if curr_iter == MAX_ITERS:
        return True
    
    return False

In [310]:
reset_population()
generate_population()
iteration = 0

curr_best_fitness = float("inf")

while (not should_stop(curr_best_fitness, iteration)):
    reset_success()
    
    filhos = crossover()
    # filhos = mutate(filhos)
    generational(filhos)

    curr_best_fitness = fitness(FITNESS_FUNC, POPULACAO[0][0])
      
    iteration += 1

    if iteration % 50 == 0:
        print(curr_best_fitness, POPULACAO[0][0])

print(iteration)


13.644285218849518 [-2.134758914249332, -5.600933938143857, 0.3345529277721191, -5.315666287577262, -5.112130993636775, 3.3965479302041253, 5.303401800277516, 5.031098990359773, 0.6334013005359456, -2.4495992028492157, 0.8311814374745996, 7.564759101949669, 8.046765631247535, -2.0423408810617065, -0.0457691648445806, -1.0377929543280509, 2.991249587576842, -4.98792263926741, -6.174099776071401, -3.3164524205953683, 6.539524977177667, 1.8675657374425612, 6.217587081941055, 4.8673141782347145, 1.8017613941705868, 2.3225198285658237, 6.123187222610834, 11.095312193843377, 1.7071194340756648, -0.15500088137774348]
13.644285218849518 [-2.134758914249332, -5.600933938143857, 0.3345529277721191, -5.315666287577262, -5.112130993636775, 3.3965479302041253, 5.303401800277516, 5.031098990359773, 0.6334013005359456, -2.4495992028492157, 0.8311814374745996, 7.564759101949669, 8.046765631247535, -2.0423408810617065, -0.0457691648445806, -1.0377929543280509, 2.991249587576842, -4.98792263926741, -6.1

KeyboardInterrupt: 

# Estratégia Genética

## Versão 1